## JeJu Road Dataset Info.

### 변수명,변수 설명

- id : 아이디
- base_date : 날짜
- day_of_week : 요일
- base_hour : 시간대
- road_in_use : 도로사용여부
- lane_count : 차로수
- road_rating : 도로등급
- multi_linked : 중용구간 여부
- connect_code : 연결로 코드
- maximum_speed_limit : 최고속도제한
- weight_restricted : 통과제한하중
- height_restricted : 통과제한높이
- road_type : 도로유형
- start_latitude : 시작지점의 위도
- start_longitude : 시작지점의 경도
- start_turn_restricted : 시작 지점의 회전제한 유무
- end_latitude : 도착지점의 위도
- end_longitude : 도착지점의 경도
- end_turn_restricted : 도작지점의 회전제한 유무
- road_name : 도로명
- start_node_name : 시작지점명
- end_node_name : 도착지점명
- vehicle_restricted : 통과제한차량
- target : 평균속도(km)

In [1]:
import pandas as pd
import numpy as np

# Read the data
test_data = pd.read_csv('./data/test.csv')
train_data = pd.read_csv('./data/train.csv')


In [2]:
train_data.head()


,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,20220623,목,17,1,106,지방도1112호선,0,0,60.0,...,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,20220728,목,21,2,103,일반국도11호선,0,0,60.0,...,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,20211010,일,7,2,103,일반국도16호선,0,0,80.0,...,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,20220311,금,13,2,107,태평로,0,0,50.0,...,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,20211005,화,8,2,103,일반국도12호선,0,0,80.0,...,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0


- '-' → NaN값으로 변경

In [3]:
train_data = train_data.replace('-', np.NaN)

- 결측치 확인

In [30]:
# Check for missing values
missing_values_train_data = train_data.isnull().sum()
# Display the missing values
print(missing_values_train_data)

id                       0
base_date                0
day_of_week              0
base_hour                0
lane_count               0
road_rating              0
road_name                0
multi_linked             0
connect_code             0
maximum_speed_limit      0
vehicle_restricted       0
weight_restricted        0
height_restricted        0
road_type                0
start_node_name          0
start_latitude           0
start_longitude          0
start_turn_restricted    0
end_node_name            0
end_latitude             0
end_longitude            0
end_turn_restricted      0
target                   0
month                    0
rough_road_name          0
line_number              0
start_to_end             0
dtype: int64


In [5]:
train_data.shape

(4701217, 23)

In [6]:
train_data.dtypes

id                        object
base_date                  int64
day_of_week               object
base_hour                  int64
lane_count                 int64
road_rating                int64
road_name                 object
multi_linked               int64
connect_code               int64
maximum_speed_limit      float64
vehicle_restricted       float64
weight_restricted        float64
height_restricted        float64
road_type                  int64
start_node_name           object
start_latitude           float64
start_longitude          float64
start_turn_restricted     object
end_node_name             object
end_latitude             float64
end_longitude            float64
end_turn_restricted       object
target                   float64
dtype: object

- NaN값 제거

In [8]:
train_data = train_data.dropna(axis=0)

### 도로명(road_name)
- 도로명은 일반국도, 지방도, -로 구성되어 있음 -> 이들을 1,2,3으로 인코딩해서 분석

->  ~로가 국도, 지방도에 비해 평균속도가 낮았다 : -로는 시내도로이기 때문인 것으로 예상

In [16]:
def change_value(x):
    if '일반국도' in x:
        return 1
    elif '지방도' in x:
        return 2
    else:
        return 3

# 'A' 컬럼에 함수 적용
train_data['rough_road_name'] = train_data['road_name'].apply(change_value)

In [33]:
grouped = train_data.groupby('rough_road_name')['target'].mean()
grouped

rough_road_name
1    46.479098
2    47.376826
3    33.249800
Name: target, dtype: float64

### 날짜(월(base_date), 계절(season), 요일(day_of_week))

- 요일별은 거의 차이가 없음: 주말과 주중이 차이 없는것으로 보아, 평균속도가 관광객의 영향이 많이 받는 것으로 보임

- 계절별로 월을 나누어 확인해 본 결과, 여름(6-8월)이 가장 평균 속도가 낮음, 나머진 비슷

- 월별로 확인해 본 결과, 성수기인 7월이 가장 평균 속도가 낮았다.(7월: 35 < 나머지: 42): 계절보다는 성수기/비성수기가 더 결정요인이 큰것같음

+) 데이터가 21.9 - 22.7의 데이터기 때문에, 8월 데이터는 확인할 수 없었음+) 테스트 데이터가 22.8월 데이터임.. <- 아마도 성수기로 봐야할 것 같음

- 요일별 데이터 분석

In [34]:
train_data['day_of_week'].replace({'월': 1, '화': 2, '수': 3, '목': 4, '금': 5, '토': 6, '일': 7,}, inplace=True)

- 날짜

In [10]:
train_data["base_date"] - 20000000

0          220623
1          220728
2          211010
3          220311
4          211005
            ...  
4701210    220105
4701211    211230
4701214    220613
4701215    211020
4701216    211019
Name: base_date, Length: 4131754, dtype: int64

In [11]:
train_data["base_date"] = train_data["base_date"] - 20000000

In [12]:
train_data["base_date"] = train_data["base_date"] % 10000

- 계절

In [37]:
def change_value(x):
    month = int(x / 100)
    if month in [12,1,2]:
        return 1
    elif month in [3,4,5]:
        return 2
    elif month in [6,7,8]:
        return 3
    else:
        return 4
train_data['season'] = train_data['base_date'].apply(change_value)

In [38]:
grouped = train_data.groupby('season')['target'].mean()
grouped

season
1    42.839088
2    42.900467
3    39.905235
4    43.426602
Name: target, dtype: float64

- 월별 데이터 분석

In [14]:
def change_value(x):
    month = int(x / 100)
    return month

# 'A' 컬럼에 함수 적용
train_data['month'] = train_data['base_date'].apply(change_value)

In [39]:
grouped = train_data.groupby('month')['target'].mean()
grouped

month
1     42.756695
2     42.993206
3     43.327702
5     42.433164
6     42.421940
7     35.814802
9     43.527451
10    43.301642
11    43.505672
12    42.777511
Name: target, dtype: float64

### 성수기/비성수기
- 7/8월 : 성수기 
- else : 비성수기

In [40]:
def change_value(x):
    month = int(x / 100)
    if month in [7,8]:
        return True
    else:
        return False
train_data['peak_season'] = train_data['base_date'].apply(change_value)

In [41]:
grouped = train_data.groupby('peak_season')['target'].mean()
grouped

peak_season
False    42.969586
True     35.814802
Name: target, dtype: float64

### 시간(base_hour)

- 10~15시가 새벽~밤 대비 평균적으로 낮은 속도를 보였고, 특히 18시에 제일 낮은 평균속도를 보임: 18시는 관광객 + 퇴근 인구 때문에 가장 낮은 평균속도를 보였을 것으로 예상: 출근시간은 크게 막히지 않는 모습을 보임: 평균속도를 결정하는 것은 주민 < 관광객

In [42]:
grouped = train_data.groupby('base_hour')['target'].mean()
grouped

base_hour
0     48.493093
1     49.488644
2     49.990755
3     50.243062
4     49.461382
5     46.907501
6     45.272625
7     43.436869
8     41.298290
9     40.526329
10    39.865881
11    39.465107
12    39.496214
13    39.390996
14    39.211991
15    38.966294
16    38.705750
17    38.028510
18    37.790192
19    39.806965
20    40.876185
21    41.911651
22    43.704757
23    45.542206
Name: target, dtype: float64

### 차선 수(lane_count)
- 평균 속도: 2(44.98)>1(43.27)>>>>>3(34.91): 차선이 많은 도로일수록 오히려 많은 관광객이 몰리는 도로일 확률이 높다.

In [43]:
grouped = train_data.groupby('lane_count')['target'].mean()
grouped

lane_count
1    43.278007
2    44.981769
3    34.917783
Name: target, dtype: float64

### 최고속도제한

- 40km 제한인 도로가 가장 평균 속도가 높음: 제한속도 40km 인 도로는 1차선에만 존재: 제한속도 40km 도로는 일반국도 12, 95 

- 특히 일반국도 95가 평균속도(70)가 높음-> 잘 안쓰는 도로일 것으로 예상됨 

- 95번 도로는 산업도로, 12번 도로는 일주도로 -> 그래서 95번이 유독 빨랐을 것으로 예상

In [44]:
grouped = train_data.groupby('maximum_speed_limit')['target'].mean()
grouped

maximum_speed_limit
30.0    29.788546
40.0    62.774080
50.0    34.991438
60.0    42.208437
70.0    40.981438
80.0    60.852437
Name: target, dtype: float64

### 호선
- 일반국도/지방도는 호선마다 평균 속도가 다름: 일반국도) 11(39.99)<99<12<16<95(70.44) 순으로 차이남

  지방도) 1120 이 가장 낮음(36)

In [17]:
train_data['line_number'] = train_data['road_name'].str.extract('(\d+)', expand=False).fillna(0).astype(int)

In [20]:
road_1 = train_data[train_data["rough_road_name"]==2] 
road_1.groupby("line_number")['target'].mean()

line_number
97      60.525723
1112    47.291909
1115    53.756296
1116    61.580721
1117    58.903308
1118    50.635763
1119    41.875178
1120    36.228093
1132    40.370720
1136    45.922186
Name: target, dtype: float64

In [21]:
road_2 = train_data[train_data["rough_road_name"]==1]
road_2["maximum_speed_limit"].value_counts()

maximum_speed_limit
60.0    692908
80.0    561535
70.0    471407
50.0    377096
30.0     37747
40.0      6790
Name: count, dtype: int64

### 위도/경도(start_end_longitude)

- 더 잘해야할 것 같음. 데이터 부족시 추가조사 필요

- 초기실험에서는 제외하는것이 좋을 것 같음(minmax scaling 필요)

- 출발지-도착지 이름도 있지만, 같은 이름이라면 위도/경도가 같기 때문에 사용이 필요할지 생각해봐야할 것 같음

In [24]:
train_data['start_longitude'].value_counts()

start_longitude
126.567766    21874
126.736249    20775
126.556173    18084
126.505998    18076
126.509864    17879
              ...  
126.682267      881
126.677154      788
126.524410      744
126.524429      744
126.525745      587
Name: count, Length: 518, dtype: int64

In [25]:
train_data['start_latitude'].value_counts()

start_latitude
33.248633    21874
33.435941    20775
33.507349    18084
33.500330    18076
33.249029    17879
             ...  
33.491729      881
33.437249      788
33.256671      744
33.261270      744
33.257401      587
Name: count, Length: 518, dtype: int64

In [35]:
train_data['start_latitude'].value_counts()

start_latitude
33.248633    21874
33.435941    20775
33.507349    18084
33.500330    18076
33.249029    17879
             ...  
33.491729      881
33.437249      788
33.256671      744
33.261270      744
33.257401      587
Name: count, Length: 518, dtype: int64

In [26]:
train_data['start_to_end'] = train_data['start_node_name'] + '-' + train_data['end_node_name']

In [29]:
train_data['start_to_end'].value_counts() 

start_to_end
미수2교-미수2교                         21294
구엄교-구엄교                           16816
부록교 시종점-부록교 시종점                   12147
봉개교 시종점-봉개교 시종점                   12145
우남육교-우남육교                         12091
                                  ...  
서귀포시 호근동 1838-4-서귀포시 서호동 308-6      744
CU서귀호근로점-서호교회                       744
서호교회-서귀포시 호근동 1838-4                744
서호교회-CU서귀호근로점                       587
식당-서호교회                             587
Name: count, Length: 697, dtype: int64

In [28]:
train_data['start_longitude'].value_counts() 

start_longitude
126.567766    21874
126.736249    20775
126.556173    18084
126.505998    18076
126.509864    17879
              ...  
126.682267      881
126.677154      788
126.524410      744
126.524429      744
126.525745      587
Name: count, Length: 518, dtype: int64

In [37]:
train_data[train_data["road_name"]=="일반국도12호선"]["start_longitude"].value_counts()

start_longitude
126.505998    12113
126.512046    12108
126.516469    12103
126.536446    12095
126.534803    12089
              ...  
126.187070     5080
126.894049     5078
126.212819     4816
126.182617     4653
126.210170     4390
Name: count, Length: 138, dtype: int64

In [45]:
train_data.keys()

Index(['id', 'base_date', 'day_of_week', 'base_hour', 'lane_count',
       'road_rating', 'road_name', 'multi_linked', 'connect_code',
       'maximum_speed_limit', 'vehicle_restricted', 'weight_restricted',
       'height_restricted', 'road_type', 'start_node_name', 'start_latitude',
       'start_longitude', 'start_turn_restricted', 'end_node_name',
       'end_latitude', 'end_longitude', 'end_turn_restricted', 'target',
       'month', 'rough_road_name', 'line_number', 'start_to_end', 'season',
       'peak_season'],
      dtype='object')

In [47]:
grouped = train_data.groupby('connect_code')['target'].mean()
grouped

connect_code
0      42.469219
103    57.947043
Name: target, dtype: float64

In [ ]:
train_data.to_csv('train.csv', index=False)